In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%ls

drive/  sample_data/


In [4]:
test = pd.read_csv("/content/drive/MyDrive/7313Project/test_reduced.csv")
train = pd.read_csv("/content/drive/MyDrive/7313Project/train_reduced.csv")
testn = pd.read_csv("/content/drive/MyDrive/7313Project/test_reduced.csv")
target = pd.read_csv("/content/drive/MyDrive/7313Project/target.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
train.drop(train.iloc[:, [0,1]], axis=1, inplace=True)
test.drop(test.iloc[:, [0,1]], axis=1, inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [7]:
train['is_cup'] = labelencoder.fit_transform(train['is_cup'])
test['is_cup'] = labelencoder.fit_transform(test['is_cup'])
target['target'].replace({"home": 0, "draw": 1, "away":2}, inplace=True)
train.drop(train.filter(regex='recorded_matches').columns, axis=1, inplace = True)
test.drop(test.filter(regex='recorded_matches').columns, axis=1, inplace = True)
train_x = train.copy()
train_y = target['target'].copy()
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

In [8]:
from sklearn.preprocessing import StandardScaler
#fit scaler on training data
norm = StandardScaler().fit(train_x)
#transform training data
xtrain_norm = norm.transform(train_x)
#transform testing data
test_norm = norm.transform(test)

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


# cv_acc = cross_val_score(estimator=RandomForestClassifier(n_estimators=50, random_state=1),
#                          X=xtrain_norm,
#                          y=train_y,
#                          cv=5,
#                          n_jobs=5)

# print('Kfold Accuracy: %.2f%%' % (np.mean(cv_acc)*100))

In [10]:
# forest = RandomForestClassifier(n_estimators=300,
#                                 random_state=1)

# forest.fit(xtrain_norm, train_y)

# y_pred = forest.predict(xtrain_norm)
# acc = np.mean(y_pred == train_y)*100
# acc

In [11]:
# from mlxtend.evaluate import confusion_matrix
# confmat = confusion_matrix(train_y, y_pred)

# print(confmat)

In [12]:
# prediction_1 = forest.predict_proba(test_norm)
# prediction_1

In [14]:
from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb
from mlxtend.evaluate import confusion_matrix
test_preds_1 = []
test_preds_2 = []

kf = StratifiedKFold(n_splits=5, random_state=123, shuffle=True) # KFold
i=0
for fold, (train_idx, test_idx) in enumerate(kf.split(xtrain_norm, train_y)):
    i=i+1
    print(i)
    X_train, X_test = xtrain_norm[train_idx], xtrain_norm[test_idx]
    y_train, y_test = train_y.iloc[train_idx], train_y.iloc[test_idx]
    model = RandomForestClassifier(n_estimators=300,
                                random_state=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    train_acc = np.mean(y_pred == y_test)*100
    confmat = confusion_matrix(y_pred, y_test)
    print(train_acc)
    print(confmat)
    test_preds_1.append(model.predict_proba(xtrain_norm).squeeze())
    test_preds_2.append(model.predict_proba(test_norm).squeeze())

1
49.22931314223905
[[7535 3665 3673]
 [ 280  266  240]
 [1808 1599 3122]]
2
48.81016765819362
[[7539 3736 3758]
 [ 245  240  226]
 [1839 1554 3051]]
3
49.63944474490716
[[7564 3673 3607]
 [ 278  256  234]
 [1780 1602 3194]]
4
49.091810519673686
[[7546 3667 3686]
 [ 248  222  225]
 [1828 1641 3124]]
5
49.03772479379817
[[7587 3660 3747]
 [ 272  260  254]
 [1764 1610 3033]]


In [15]:
from mlxtend.evaluate import confusion_matrix
predictions = sum(test_preds_1)/5
classes = np.argmax(predictions, axis = 1)
print(classes)
y_true = train_y.squeeze()
confmat = confusion_matrix(y_true, classes)

print(confmat)

[2 0 1 ... 1 2 0]
[[48107     0     6]
 [   74 27574     3]
 [  125     0 35049]]


In [16]:
train_acc = np.mean(classes == y_true)*100
train_acc

99.81250788728839

In [17]:
prediction_1 = sum(test_preds_2)/5
prediction_1

array([[0.32666667, 0.368     , 0.30533333],
       [0.38733333, 0.268     , 0.34466667],
       [0.43933333, 0.24466667, 0.316     ],
       ...,
       [0.3       , 0.28933333, 0.41066667],
       [0.514     , 0.29066667, 0.19533333],
       [0.34266667, 0.30533333, 0.352     ]])

In [18]:
submission = pd.concat([pd.DataFrame(testn['id']), pd.DataFrame(prediction_1, columns=['home', 'draw', 'away'])[['home', 'draw', 'away']]], axis=1)
submission.to_csv('/content/drive/MyDrive/7313Project/submission_forest.csv', index=False)